In [1]:
%%capture
!pip install pycocotools
!pip install open_clip_torch

In [2]:
import torch
from torch.utils.data import Dataset
from torchvision.datasets import CocoDetection

import cv2
from PIL import Image

from transformers import AutoTokenizer
# import open_clip

from tqdm.auto import tqdm
import json
import sys

In [3]:
from pycocotools.coco import COCO

# чекаем данные

In [13]:
caption_path = '/kaggle/input/coco-captions-val/captions_val2014.json'
with open(caption_path, 'r') as f:
    q = list(f)[0]
    captions = json.loads(q)

In [18]:
captions['images'][0]

{'license': 3,
 'file_name': 'COCO_val2014_000000391895.jpg',
 'coco_url': 'http://images.cocodataset.org/val2014/COCO_val2014_000000391895.jpg',
 'height': 360,
 'width': 640,
 'date_captured': '2013-11-14 11:18:45',
 'flickr_url': 'http://farm9.staticflickr.com/8186/8119368305_4e622c8349_z.jpg',
 'id': 391895}

In [30]:
for i, caption_s in tqdm(enumerate(captions)):
    for j in range(len(captions[caption_s])):
#         query = "Что изображено на данной картинке?"
#         answer = captions[caption_s][j]
        print(captions[caption_s]если A^3 = 0, значит обратная к A^2 это A, caption_s)
#         self.img_paths.append(caption_s)
#         self.query_tokens.append(torch.tensor(self.tokenizer.encode(query), dtype=torch.int64))
#         self.answer_tokens.append(torch.tensor(self.tokenizer.encode(answer), dtype=torch.int64))
#         max_seq_len = max(max_seq_len, self.answer_tokens[-1].shape[0])
#     if i >= max_img:
#         break
        break
    break

0it [00:00, ?it/s]

{'description': 'COCO 2014 Dataset', 'url': 'http://cocodataset.org', 'version': '1.0', 'year': 2014, 'contributor': 'COCO Consortium', 'date_created': '2017/09/01'} info


In [23]:
for i, caption in tqdm(enumerate(captions)):
    for j in range(len(captions[caption])):
#         query = "Что изображено на картинке?"
        print(caption, caption[j])#, captions[caption][j])
#         answer = captions_raw[caption
#         self.img_paths += [caption]
        #почему не сделать tokenizer.encode(query, padding=True, truncation=True) ?
#         self.query_tokens += [torch.tensor(self.tokenizer.encode(query), dtype=torch.int64)]
#         self.answer_tokens += [torch.tensor(self.tokenizer.encode(answer), dtype=torch.int64)]
        break
    break

0it [00:00, ?it/s]

info i


In [ ]:
class CocoDataset(Dataset):
    def __init__(self, config, image_path, ann_file, caption_path, data_type="train", coef_size=0.1,
                 tokenizer_name="", prefix_length=20, normalize_prefix=False):
        if not tokenizer_name:
            tokenizer_name = config.decoder
        self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
        clip_model, _, self.preprocess = open_clip.create_model_and_transform(config.encoder, pretrained="laion400m_e32")
        self.prefix_length = prefix_length
        self.normalize_prefix = normalize_prefix
        self.img_path = image_path
        self.ann_file = ann_file
        self.caption_path = caption_path
        
        with open(caption_path, 'r') as f:
            captions_raw = json.loads(list(f)[0])
        
        self.img_paths = []
        self.query_tokens = []
        self.answer_tokens = []
        
        
        max_img = len(captions_raw)*coef_size 
        for i, caption in tqdm(enumerate(captions_raw), total=max_img):
            for j in range(len(captions_raw[caption])):
                query = "Что изображено на картинке?"
                answer = captions_raw[caption][j]
                self.img_paths += [caption]
                #почему не сделать tokenizer.encode(query, padding=True, truncation=True) ?
                self.query_tokens += [torch.tensor(self.tokenizer.encode(query), dtype=torch.int64)]
                self.answer_tokens += [torch.tensor(self.tokenizer.encode(answer), dtype=torch.int64)]
            if i >= max_img:
                break
        del captions_raw
        sys.stdout.flush()
        
        #all_len
        self.max_seq_len = prefix_length
        self.type = data_type
        
    """Почему не паддили captions?"""
    def pad_tokens(self, item: int):
        query_tokens = self.query_tokens[item]
        padding = self.max_seq_len - query_tokens.shape[0]
        if padding > 0:
            query_tokens = torch.cat((query_tokens, torch.zeros(padding, dtype=torch.int64) - 1))
            self.query_tokens[item] = query_tokens
        elif padding < 0:
            query_tokens = query_tokens[:self.max_seq_len]
            self.query_tokens[item] = query_tokens
        query_mask = tokens.ge(0)  # mask is zero where we out of sequence
        query_tokens[~query_mask] = 0
        query_mask = query_mask.float()

        answer_tokens = self.answer_tokens[item]
        padding = self.max_seq_len - answer_tokens.shape[0]
        if padding > 0:
            answer_tokens = torch.cat((answer_tokens, torch.zeros(padding, dtype=torch.int64) - 1))
            self.answer_tokens[item] = answer_tokens
        elif padding < 0:
            answer_tokens = answer_tokens[:self.max_seq_len]
            self.answer_tokens[item] = answer_tokens
        answer_mask = tokens.ge(0)  # mask is zero where we out of sequence
        answer_tokens[~answer_mask] = 0
        answer_mask = answer_mask.float()
        
        return query_tokens, query_mask, answer_tokens, answer_mask
    
    def get_image(self, item):
        name = str(self.img_paths[item])
        name = f"{self.img_path}/{name}"
        image_resized = cv2.resize(cv2.imread(naem), (256,256))
        return Image.fromarray(cv2.cvtColor(image_resized, cv2.COLOR_BGR2RGB))

    def __len__(self) -> int:
        return len(self.img_paths)

    def __getitem__(self, item):
#         tokens, mask = self.pad_tokens(item)
#         prefix = self.prefixes[item]
#         if self.normalize_prefix:
#             prefix = prefix.float()
#             prefix = prefix / prefix.norm(2, -1)
#         return tokens, mask, prefix
        image = self.get_image(item)
        image = self.preprocess(image).unsqueeze(0)
        query_tokens, query_mask, answer_tokens, answer_mask = pad_tokens(item)
        return query_tokens, query_mask, answer_tokens, answer_mask, image[0], item
    def show_image(self, item):
        image = self.get_image(item)
        text = self.tokenizer.decode(self.pad_tokens(item)[2])
        plt.imshow(img)
        print(text)

In [14]:
from datasets import load_dataset

In [19]:
vqa2 = load_dataset("HuggingFaceM4/VQAv2")

Repo card metadata block was not found. Setting CardData to empty.


In [20]:
train_vqa2 = vqa2["train"]

In [ ]:
# train_vqa2.set_format("pandas")

In [21]:
instance = train_vqa2[1]

In [12]:
train_vqa2["question"][:5]

['What is this photo taken looking through?',
 'What position is this man playing?',
 'What color is the players shirt?',
 'Is this man a professional baseball player?',
 'What color is the snow?']

In [22]:
instance

{'question_type': 'what',
 'multiple_choice_answer': 'pitcher',
 'answers': [{'answer': 'pitcher', 'answer_confidence': 'yes', 'answer_id': 1},
  {'answer': 'catcher', 'answer_confidence': 'no', 'answer_id': 2},
  {'answer': 'pitcher', 'answer_confidence': 'yes', 'answer_id': 3},
  {'answer': 'pitcher', 'answer_confidence': 'yes', 'answer_id': 4},
  {'answer': 'pitcher', 'answer_confidence': 'yes', 'answer_id': 5},
  {'answer': 'pitcher', 'answer_confidence': 'yes', 'answer_id': 6},
  {'answer': 'pitcher', 'answer_confidence': 'yes', 'answer_id': 7},
  {'answer': 'pitcher', 'answer_confidence': 'yes', 'answer_id': 8},
  {'answer': 'pitcher', 'answer_confidence': 'yes', 'answer_id': 9},
  {'answer': 'pitcher', 'answer_confidence': 'yes', 'answer_id': 10}],
 'image_id': 458752,
 'answer_type': 'other',
 'question_id': 458752001,
 'question': 'What position is this man playing?',
 'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=640x480>}

In [ ]:
class VQAv2Dataset(Dataset):
    """Датасет для файнтюна"""
    def __init__(self, split, processor):
        if (split == "train"):
            self.dataset = load_dataset("HuggingFaceM4/VQAv2", split="train[:90%]")
        elif (split == "val"):
            self.dataset = load_dataset("HuggingFaceM4/VQAv2", split="train[90%:]")
#         self.dataset = dataset
        self.processor = processor

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, item):
        question = self.dataset[item]['question']
        answer = self.dataset[item]['answer']
        image_id = self.dataset[item]['pid']
        image_path = f"Data/train_fill_in_blank/{image_id}/image.png"
        image = Image.open(image_path).convert("RGB")
        text = question
        
        encoding = self.processor(image, text, padding="max_length", truncation=True, return_tensors="pt")
        labels = self.processor.tokenizer.encode(
            answer, max_length= 8, pad_to_max_length=True, return_tensors='pt'
        )
        encoding["labels"] = labels

        for k,v in encoding.items():  encoding[k] = v.squeeze()
        return encoding

In [43]:
# instance['image']

In [37]:
answers = train_vqa2["multiple_choice_answer"]
questions = train_vqa2["question"]
# images = train_vqa2["image"]


KeyboardInterrupt



In [35]:
for i, answer in tqdm(enumerate(answers)):
    print(i, answer, questions[i], answers[i])
    break

0it [00:00, ?it/s]

0 net What is this photo taken looking through? net


In [ ]:
class VQAv2_Dataset(Dataset):
    def __init__(self, config, dataset, coef_size=0.1,
                 tokenizer_name="", prefix_length=20, normalize_prefix=False):
        if not tokenizer_name:
            tokenizer_name = config.decoder
        self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
        clip_model, _, self.preprocess = open_clip.create_model_and_transform(config.encoder, pretrained="laion400m_e32")
        self.prefix_length = prefix_length
        self.normalize_prefix = normalize_prefix
        self.dataset = datatset  #load_dataset("HuggingFaceM4/VQAv2", split=split)
#         self.img_path = image_path
#         self.ann_file = ann_file
#         self.caption_path = caption_path
#         coco_dataset_raw = CocoDetection(root=self.img_path, annFile=self.ann_file)
        
#         with open(caption_path, 'r') as f:
#             captions_raw = json.loads(list(f)[0])
        answers = dataset["multiple_choice_answer"]
        questions = dataset["question"]
        self.image_idx = []
        self.query_tokens = []
        self.answer_tokens = []
        
        max_img = len(answers)*coef_size 
        for i, answer in tqdm(enumerate(answers), total=max_img):
            query = questions[i]
            self.query_tokens += [torch.tensor(self.tokenizer.encode(query), dtype=torch.int64)]
            self.answer_tokens += [torch.tensor(self.tokenizer.encode(answer), dtype=torch.int64)]
            self.images += i
            if i >= max_img:
            break
        del captions_raw
        sys.stdout.flush()
        
        #all_len
        self.max_seq_len = prefix_length
        self.type = data_type
        
    """Почему не паддили captions?"""
    def pad_tokens(self, item: int):
        query_tokens = self.query_tokens[item]
        padding = self.max_seq_len - query_tokens.shape[0]
        if padding > 0:
            query_tokens = torch.cat((query_tokens, torch.zeros(padding, dtype=torch.int64) - 1))
            self.query_tokens[item] = query_tokens
        elif padding < 0:
            query_tokens = query_tokens[:self.max_seq_len]
            self.query_tokens[item] = query_tokens
        query_mask = tokens.ge(0)  # mask is zero where we out of sequence
        query_tokens[~query_mask] = 0
        query_mask = query_mask.float()


        answer_tokens = self.answer_tokens[item]
        padding = self.max_seq_len - answer_tokens.shape[0]
        if padding > 0:
            answer_tokens = torch.cat((answer_tokens, torch.zeros(padding, dtype=torch.int64) - 1))
            self.answer_tokens[item] = answer_tokens
        elif padding < 0:
            answer_tokens = answer_tokens[:self.max_seq_len]
            self.answer_tokens[item] = answer_tokens
        answer_mask = tokens.ge(0)  # mask is zero where we out of sequence
        answer_tokens[~answer_mask] = 0
        answer_mask = answer_mask.float()
        
        return query_tokens, query_mask, answer_tokens, answer_mask
    
    def get_image(self, item):
#         name = str(self.img_paths[item])
#         name = f"{self.img_path}/{name}"
#         image_resized = cv2.resize(cv2.imread(naem), (256,256))
#         return Image.fromarray(cv2.cvtColor(image_resized, cv2.COLOR_BGR2RGB))
        image_resized = cv2.resize(self.image_idx[item], (256,256))
        return Image.fromarray(cv2.cvtColor(image_resized, cv2.COLOR_BGR2RGB))

    def __len__(self) -> int:
        return len(self.image_idx)

    def __getitem__(self, item):
        image = self.get_image(item)
        image = self.preprocess(image).unsqueeze(0)
        query_tokens, query_mask, answer_tokens, answer_mask = pad_tokens(item)
        return query_tokens, query_mask, answer_tokens, answer_mask, image[0], item
    def show_image(self, item):
        image = self.get_image(item)
        text = self.tokenizer.decode(self.pad_tokens(item)[2])
        plt.imshow(img)
        print(text)